In [20]:
from PIL import Image
from transformers import AutoProcessor, Kosmos2ForConditionalGeneration
import pandas as pd
import time

model = Kosmos2ForConditionalGeneration.from_pretrained("microsoft/kosmos-2-patch14-224")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#Daten einlesen
df_aktbilder = pd.read_csv("../../data/filtered_genres_nude_painting.csv")
default_path = "../../data/zielordner_nude_painting/"

prompt_result = []
start = time.time()

for filename in list(df_aktbilder.file_name)[:2]: #[1000:2000]
    try: 
        path = default_path + filename
        image = Image.open(path)
    
    
        prompt = "<grounding> Identify the number of people in the image. Describe their individual body posture and what they are doing. Also, provide a gender assessment for each person."
        inputs = processor(text=prompt, images=image, return_tensors= "pt")
        
        generated_ids = model.generate(
            pixel_values=inputs["pixel_values"],
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            image_embeds=None,
            image_embeds_position_mask=inputs["image_embeds_position_mask"],
            use_cache=True,
            max_new_tokens=150,
        )
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        caption, entities = processor.post_process_generation(generated_text) 
        print(len(caption.split(" ")))
        
        result = [filename, prompt.replace("<grounding> ",""), caption.replace(prompt.replace("<grounding> ",""),"")] 
        prompt_result.append(result)
    except Exception as e:
        print(f"Fehler bei Datei: {filename}")

end = time.time()
elapsed = end - start
print(f"Elapsed: {elapsed} seconds")

In [ ]:
print(prompt_result)
df = pd.DataFrame(prompt_result)
df.to_csv("/Users/int-veen/Documents/CulturalAnalytics/src/kosmos/prompt_result.csv")

In [ ]:
#from PIL import ImageDraw
#for label, sentence_pos, coordinates in entities:
#  for a,b,c,d in coordinates:
#      draw = ImageDraw.Draw(image)
#      w,h = image.size
#      draw.text((a*w+5,b*h+5),label + f" {sentence_pos}", fill="black")
#      draw.text((a*w+4,b*h+4),label + f" {sentence_pos}", fill="white")
#      draw.rectangle((a*w,b*h,c*w,d*h), outline ="red", width=3)
#      draw.rectangle((a*w,b*h,c*w,d*h), outline ="white", width=2)
#      draw.rectangle((a*w,b*h,c*w,d*h), outline ="black", width=1)
#image.show()
#print(caption)